* 본 ipynb 파일은 python 3.9.6 64bit 환경에서 실행됨

# 필요한 패키지 import

In [42]:
import re
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats

plt.rc('font',family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)
set_matplotlib_formats('retina')
%matplotlib inline

pd.set_option('float_format', '{:.0f}'.format)

C:\Users\user\AppData\Local\Temp/ipykernel_145936/367198997.py:12: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina')


In [53]:
L = os.listdir(r"D:\명지대학교\비교과\공모전 및 대외활동\재정데이터 시각화 경진대회\추가 조사한 데이터\사업별 예산 시계열")
for i in L:
    if not i.endswith(".xls"):
        L.remove(i)
L

['감사원.xls',
 '개인정보보호위원회.xls',
 '경찰청 (2).xls',
 '고용노동부 (2).xls',
 '공정거래위원회 (2).xls',
 '과학기술정보통신부.xls',
 '관세청.xls',
 '교육부.xls',
 '국가보훈처.xls',
 '국가인권위원회.xls',
 '국방부.xls',
 '국세청.xls',
 '국토교통부.xls',
 '국회.xls',
 '금융위원회.xls',
 '기상청.xls',
 '기획재정부.xls',
 '농림축산식품부.xls',
 '농촌진흥청.xls',
 '대법원.xls',
 '대통령경호처.xls',
 '대통령비서실 및 국가안보실.xls',
 '문화재청.xls',
 '문화체육관광부.xls',
 '민주평화통일자문회의.xls',
 '방송통신위원회.xls',
 '방위사업청.xls',
 '법무부.xls',
 '법제처.xls',
 '병무청.xls',
 '보건복지부.xls',
 '산림청.xls',
 '산업통상자원부.xls',
 '새만금개발청.xls',
 '소방청.xls',
 '식품의약품안전처.xls',
 '여성가족부.xls',
 '외교부.xls',
 '원자력안전위원회.xls',
 '인사혁신처.xls',
 '조달청.xls',
 '중소벤처기업부.xls',
 '중소벤처사업부.xls',
 '중앙선거관리위원회.xls',
 '질병관리청.xls',
 '통계청.xls',
 '통일부.xls',
 '특허청.xls',
 '해양경찰청.xls',
 '해양수산부.xls',
 '행정안전부.xls',
 '행정중심복합도시건설청.xls',
 '헌법재판소.xls',
 '환경부.xls']

In [66]:
df_L = []
for i in L:
    df_L.append(pd.read_excel(r"D:\명지대학교\비교과\공모전 및 대외활동\재정데이터 시각화 경진대회\추가 조사한 데이터\사업별 예산 시계열"+f'\\{i}'))

In [67]:
df_raw = pd.concat(df_L, axis=0).reset_index(drop=True)

# 연도 별 column과 예산으로 데이터 정제하기

In [72]:
col_years = [i for i in df_raw.columns if i.startswith("20")]

#연도별로 자르고 예산 순위별로 나눈 것
list_df_20 = []
for c in col_years:
    df_20 = df_raw.sort_values(by=c, ascending=False)
    #당해 최종 예산만 컬럼으로 집어넣기
    df_20 = pd.concat([df_20.iloc[:, :list(df_20.columns).index(col_years[0])],df_20[c]], axis=1) 
    # '2018년 최종 예산'와 같은 글에서 숫자만 추출
    year = re.sub(r'[^0-9]', '', c)
    # 최종예산으로 이름변경
    df_20.rename(columns={c:"최종예산"}, inplace=True)
    df_20 = pd.concat([df_20.loc[:,:"세부사업"], pd.DataFrame({"연도":[int(year)]*len(df_20)}), df_20["최종예산"]], axis=1)
    list_df_20.append(df_20)
df_sorted_by_year_budget = pd.concat(list_df_20).dropna(subset=["최종예산"]).reset_index(drop=True)
#2017년부터의 데이터만 나오도록 정제하기
df_sorted_by_year_budget = df_sorted_by_year_budget.query("연도 > 2016").sort_values(by="연도").reset_index(drop=True)
df_sorted_by_year_budget

,소관,회계,계정,분야,부문,프로그램,단위사업,세부사업,연도,최종예산
0,감사원,일반회계,NaN,일반・지방행정,일반행정,감사활동 및 행정지원,수입대체경비,감사교육원 수입대체경비,2017,358000000
1,방위사업청,일반회계,NaN,국방,방위력개선,정밀타격/신특수무기사업,항공유도무기사업,중거리GPS유도키트,2017,33543000000
2,방위사업청,일반회계,NaN,국방,방위력개선,정밀타격/신특수무기사업,항공유도무기사업,장거리 공대지유도탄,2017,135902000000
3,방위사업청,일반회계,NaN,국방,방위력개선,정밀타격/신특수무기사업,항공유도무기사업,합동원거리공격탄(JASSM급),2017,0
4,방위사업청,일반회계,NaN,국방,방위력개선,신특수유도사업사업,정밀타격유도무기,합동원거리공격탄(JASSM급),2017,0
...,...,...,...,...,...,...,...,...,...,...
112740,기획재정부,일반회계,NaN,일반・지방행정,재정・금융,기획재정 행정지원,본부 기본경비,총액인건비 비대상기본경비(정책조정국),2021,396000000
112741,기획재정부,일반회계,NaN,일반공공행정,재정・금융,기획재정 행정지원,본부 기본경비,총액인건비 비대상기본경비(정책조정국),2021,0
112742,기획재정부,일반회계,NaN,일반・지방행정,재정・금융,기획재정 행정지원,본부 기본경비,총액인건비 비대상기본경비(대외경제국),2021,655000000
112743,기획재정부,일반회계,NaN,일반・지방행정,재정・금융,기획재정 행정지원,본부 기본경비,총액인건비 대상기본경비(세제실),2021,652000000


# 구글 분석 링크 열로 붙이기

In [73]:
df = df_sorted_by_year_budget 
google = 'https://www.google.com/search?q='
google_link = []

df_detail = df['세부사업']

for i in df_detail:
    g_link = google + i
    google_link.append(g_link)
df['googlelink'] = google_link
df

,소관,회계,계정,분야,부문,프로그램,단위사업,세부사업,연도,최종예산,googlelink
0,감사원,일반회계,NaN,일반・지방행정,일반행정,감사활동 및 행정지원,수입대체경비,감사교육원 수입대체경비,2017,358000000,https://www.google.com/search?q=감사교육원 수입대체경비
1,방위사업청,일반회계,NaN,국방,방위력개선,정밀타격/신특수무기사업,항공유도무기사업,중거리GPS유도키트,2017,33543000000,https://www.google.com/search?q=중거리GPS유도키트
2,방위사업청,일반회계,NaN,국방,방위력개선,정밀타격/신특수무기사업,항공유도무기사업,장거리 공대지유도탄,2017,135902000000,https://www.google.com/search?q=장거리 공대지유도탄
3,방위사업청,일반회계,NaN,국방,방위력개선,정밀타격/신특수무기사업,항공유도무기사업,합동원거리공격탄(JASSM급),2017,0,https://www.google.com/search?q=합동원거리공격탄(JASSM급)
4,방위사업청,일반회계,NaN,국방,방위력개선,신특수유도사업사업,정밀타격유도무기,합동원거리공격탄(JASSM급),2017,0,https://www.google.com/search?q=합동원거리공격탄(JASSM급)
...,...,...,...,...,...,...,...,...,...,...,...
112740,기획재정부,일반회계,NaN,일반・지방행정,재정・금융,기획재정 행정지원,본부 기본경비,총액인건비 비대상기본경비(정책조정국),2021,396000000,https://www.google.com/search?q=총액인건비 비대상기본경비(...
112741,기획재정부,일반회계,NaN,일반공공행정,재정・금융,기획재정 행정지원,본부 기본경비,총액인건비 비대상기본경비(정책조정국),2021,0,https://www.google.com/search?q=총액인건비 비대상기본경비(...
112742,기획재정부,일반회계,NaN,일반・지방행정,재정・금융,기획재정 행정지원,본부 기본경비,총액인건비 비대상기본경비(대외경제국),2021,655000000,https://www.google.com/search?q=총액인건비 비대상기본경비(...
112743,기획재정부,일반회계,NaN,일반・지방행정,재정・금융,기획재정 행정지원,본부 기본경비,총액인건비 대상기본경비(세제실),2021,652000000,https://www.google.com/search?q=총액인건비 대상기본경비(세제실)


----

# 행별 세부사업 연도 총합 순위 top 10

In [82]:
col_years = [i for i in df_raw.columns if i.startswith("20")]

series_years_row_sum = df_raw[col_years].sum(axis=1)#.map(lambda x: str(x)) #행별연도총합
series_years_row_sum.name = "행별연도총합"
df_yearsum = pd.concat([df_raw.drop(columns=col_years+['계정', '소관']), series_years_row_sum], axis=1)\
    .sort_values(by="행별연도총합", ascending=False)\
    .reset_index(drop=True).head(10)
df_yearsum

,회계,분야,부문,프로그램,단위사업,세부사업,행별연도총합
0,공공자금관리기금,일반공공행정,재정・금융,차입금원금상환,국고채원금상환(공공자금관리기금),국고채원금상환,301641097000000
1,일반회계,교육,유아및초중등교육,지방교육재정교부금,보통교부금,보통교부금,295960123000000
2,외국환평형기금,일반・지방행정,재정・금융,여유자금운용,여유자금운영(외국환평형기금),한국은행예치,283468354000000
3,공공자금관리기금,일반・지방행정,재정・금융,차입금원금상환,국고채원금상환(공공자금관리기금),국고채원금상환,265634226500000
4,공공자금관리기금,일반・지방행정,재정・금융,차입금등원금상환,국고채원금상환(공공자금관리기금),국고채원금상환,261663177000000
5,공공자금관리기금,일반・지방행정,재정・금융,회계기금간거래(예탁금),일반회계예탁(공공자금관리기금),일반회계 예탁,244287448000000
6,공공자금관리기금,일반・지방행정,재정・금융,기금간거래(예탁금),기타기금예탁(공공자금관리기금),공자기금에서 기타기금으로 예탁금(총괄계정),231759698000000
7,국민연금기금,사회복지,공적연금,여유자금운용,여유자금운용(국민연금기금),국민연금기금 국채외채권매입,230747239000000
8,일반회계,일반・지방행정,지방행정・재정지원,지방교부세,보통교부세,보통교부세,185220917000000
9,공공자금관리기금,일반・지방행정,재정・금융,기금간거래(예탁금),기타기금예탁(공공자금관리기금),기타기금 예탁,160959304000000


# 한글 원 단위로 보기

## 한글 숫자 단위 생성 함수 정의

In [81]:
#한글 숫자 단위 생성기(원 계산기)
def num2han(n = int()):
    n = int(n)
    result = ""
    han=list("만억조경")
    if n > 10**72:
        r = [num+"," if ind%4 == 0 else num for ind, num in enumerate(str(n)[::-1])][::-1]
        result += "".join(r)
        n -= (n // 10**(72*4))*(10**(72*4))

    for i in range(len(han),0,-1):
        if n >= 10**(i*4):
            result += f"{n // 10**(i*4)}{han[i-1]} "
            n -= (n //10**(i*4))*(10**(i*4))
    if n != 0:
        result += str(n)
    return result.strip()+"원"

## 원 단위 생성 함수 사용하여 표현하기

In [83]:
df_yearsum["행별연도총합"] = df_yearsum["행별연도총합"].apply(num2han)
df_yearsum

,회계,분야,부문,프로그램,단위사업,세부사업,행별연도총합
0,공공자금관리기금,일반공공행정,재정・금융,차입금원금상환,국고채원금상환(공공자금관리기금),국고채원금상환,301조 6410억 9700만원
1,일반회계,교육,유아및초중등교육,지방교육재정교부금,보통교부금,보통교부금,295조 9601억 2300만원
2,외국환평형기금,일반・지방행정,재정・금융,여유자금운용,여유자금운영(외국환평형기금),한국은행예치,283조 4683억 5400만원
3,공공자금관리기금,일반・지방행정,재정・금융,차입금원금상환,국고채원금상환(공공자금관리기금),국고채원금상환,265조 6342억 2650만원
4,공공자금관리기금,일반・지방행정,재정・금융,차입금등원금상환,국고채원금상환(공공자금관리기금),국고채원금상환,261조 6631억 7700만원
5,공공자금관리기금,일반・지방행정,재정・금융,회계기금간거래(예탁금),일반회계예탁(공공자금관리기금),일반회계 예탁,244조 2874억 4800만원
6,공공자금관리기금,일반・지방행정,재정・금융,기금간거래(예탁금),기타기금예탁(공공자금관리기금),공자기금에서 기타기금으로 예탁금(총괄계정),231조 7596억 9800만원
7,국민연금기금,사회복지,공적연금,여유자금운용,여유자금운용(국민연금기금),국민연금기금 국채외채권매입,230조 7472억 3900만원
8,일반회계,일반・지방행정,지방행정・재정지원,지방교부세,보통교부세,보통교부세,185조 2209억 1700만원
9,공공자금관리기금,일반・지방행정,재정・금융,기금간거래(예탁금),기타기금예탁(공공자금관리기금),기타기금 예탁,160조 9593억 400만원
